In [1]:
import os
import io
import time
import math
import numpy as np
from numpy import linalg
import matplotlib.pyplot as plt

# from arithmetics import *
import arithmetics
import supp_enum


np.set_printoptions(precision=4, suppress=True)
%matplotlib inline

In [2]:
def boundary(A, b, C, d, X, eps=1e-6):
    D, N = X.shape
    
    Slack = b @ np.ones([1, N]) - A @ X
    Binding = Slack <= eps
    Slack[Binding] = 0
    
    dX = np.random.randn(D, N)
    for j in range(N):
        Norm = np.concatenate([A[Binding[:, j], :].T, C.T], axis=1)
        bindcoef = linalg.pinv(np.matrix(Norm.T @ Norm)) @ Norm.T @ dX[:, j:j+1]
        dX[:, j:j+1] -= Norm @ bindcoef
    dX_good_numerical = (dX**2).sum(axis=0) >= eps ** 2
    
    dSlack = A @ dX
    dSlack[Binding] = 0
    
    Stride = Slack / dSlack
    Stride[dSlack <= 0] = np.inf
    Stride = Stride.min(axis=0)
    
    Y = X + dX * Stride
    Y = Y[:, dX_good_numerical]
    # Since Delta[:, j] // A[Flag[:, j], :].T, those binding inequations will no longer limiting the stride.
    # What still limiting the strides are only the slacking inequations.
    # And, Stride limits from binding inequations are 0/0, or at least poor numerical quality eps/eps,
    # whose inclusion will make false limits in stride length.
    # remark: g.T @ x <= h: slacking, binding, broken (NEW, g.T @ x - h > eps)
    
    return Y

In [3]:
def P_Q_sampling(A, B, eps = 1e-4, Xpop=10000, Ypop=10000):
    m, n = A.shape
    
    x = np.random.rand(m, Xpop)
    xflagsM = x >= 0 - eps
    xflagsN = (B.T @ x) <= 1 + eps
    xflag = (x >= 0 - eps).sum(axis=0) + xflagsN.sum(axis=0) == m + n
    x = x[:, xflag]
    X = [x]
    
    conv_A = np.concatenate([-np.eye(m), B.T], axis=0)
    conv_b = np.concatenate([-np.zeros([m, 1]), np.ones([n, 1])])
    conv_C = np.zeros([0, m])
    conv_d = np.zeros([0, 1])
    for i in range(m):
        X.append(boundary(conv_A, conv_b, conv_C, conv_d, X[i]))
    
    y = np.random.rand(n, Ypop)
    yflagsM = A @ y <= 1 + eps
    yflagsN = y >= 0 - eps
    yflag = yflagsM.sum(axis=0) + yflagsN.sum(axis=0) == m + n
    y = y[:, yflag]
    Y = [y]
    
    conv_A = np.concatenate([A, -np.eye(n)], axis=0)
    conv_b = np.concatenate([np.ones([m, 1]), -np.zeros([n, 1])])
    conv_C = np.zeros([0, n])
    conv_d = np.zeros([0, 1])
    for j in range(n):
        Y.append(boundary(conv_A, conv_b, conv_C, conv_d, Y[j]))
    
    return X, Y

In [4]:
class Node(object):
    MEM = np.empty(0, dtype=object)  # logical heap for type Node
    
    def __init__(self):
        # tree-node basic
        self.parent = 0
        self.just_happend = np.empty(2, dtype=int)
        self.u = 0.0
        self.v = 0.0
        self.children = np.empty([0, 0], dtype=int)
        
        # InformationSet emulated
        self.actA = np.empty([0], dtype=int)
        self.actB = np.empty([0], dtype=int)
        self.x = np.empty(0)
        self.y = np.empty(0)
        
        # logical pointer mechanism
        Node.MEM = np.append(Node.MEM, self)
        self.address = Node.MEM.shape[0] - 1
        return
    
    def __repr__(self):
        # [print(d) for d in self.__dict__]
        out = "{&: " + str(self.address) + ",\n"
        out += "parent: " + str(self.parent) + ",\n"
        out += "just_happened: " + str(self.just_happend) + ",\n"
        out += "u: " + str(self.u) + ",\n"
        out += "v: " + str(self.v) + ",\n"
        out += "actA: " + str(self.actA) + ",\n"
        out += "actB: " + str(self.actB) + ",\n"
        out += "x: " + str(self.x) + ",\n"
        out += "y: " + str(self.y) + ",\n"
        out += "children:\n" + str(self.children) + "}"
        return out
    
    def __str__(self):
        return self.__repr__()
    
    def __call__(self, ij_seq):
        node = self
        for ij in ij_seq:
            i, j = ij
            idx = np.where(node.actA == i)
            jdx = np.where(node.actB == j)
            if len(idx[0]) and len(jdx[0]):
                node = Node.MEM[node.children[idx[0][0], jdx[0][0]]]
            else:
                return Node.MEM[0]
        return node
    
    def h(self):
        n = self
        y = np.empty([0, 2], dtype=int)
        while n.parent:
            y = np.append(y, n.just_happend[None, :], axis=0)
            n = Node.MEM[n.parent]
        y = np.flip(y)
        y = y.tolist()
        return y
    
Node_NULL = Node()
Node_NULL.just_happend[:] = np.array([0, 0])

In [5]:
class GAME(object):
    M = 3
    N = 3
    UPCARD = [1, 1, 1, 1]
    valueA = [5.00, 1.33, 2.71, 1.80]
    valueB = [4.12, 6.28, 3.33, 1.92]
    A = np.empty([M, N], dtype=object)
    B = np.empty([M, N], dtype=object)
    def __init__(self):
        # Namespace does not instantiate
        return
    
    @classmethod
    def tree_generate(cls):
        root = Node()
        root.parent = 0
        root.just_happend = np.array([0, 0])
        root.actA = np.arange(GAME.M) + 1
        root.actB = np.arange(GAME.N) + 1
        root.x = arithmetics.rand_f(1, len(root.actA)-1)[0]
        root.y = arithmetics.rand_f(1, len(root.actB)-1)[0]

        bfs_layers = []
        parent_layer = [root.address]
        child_layer = []
        while True:
            bfs_layers.append(parent_layer)
            for ptr in parent_layer:
                node = Node.MEM[ptr]
                node.children.resize([len(node.actA), len(node.actB)])
                for i in range(len(node.actA)):
                    for j in range(len(node.actB)):
                        child = Node()
                        node.children[i, j] = child.address
                        child.parent = node.address
                        child.just_happend = np.array([node.actA[i], node.actB[j]])
                        child.actA = node.actA[node.actA != node.actA[i]]
                        child.actB = node.actB[node.actB != node.actB[j]]
                        child.x = arithmetics.rand_f(1, len(child.actA)-1)[0]
                        child.y = arithmetics.rand_f(1, len(child.actB)-1)[0]
                        child_layer.append(child.address)
            if len(child_layer) == 0:
                break
            else:
                parent_layer = child_layer
                child_layer = []
        return bfs_layers
    
    @classmethod
    def leaves_payoff(cls, bfs_layers):
        for ptr in bfs_layers[-1]:
            leaf = Node.MEM[ptr]
            h = leaf.h()
            u, v = 0, 0
            for i in range(len(h)):
                p, q = h[i]
                if p > q:
                    u += cls.UPCARD[i] * cls.valueA[i]
                    v -= cls.UPCARD[i] * cls.valueB[i]
                elif p < q:
                    u -= cls.UPCARD[i] * cls.valueA[i]
                    v += cls.UPCARD[i] * cls.valueB[i]
            leaf.u, leaf.v = u, v
        return

In [6]:
layers = GAME.tree_generate()

root = Node.MEM[layers[0][0]]
for i in range(len(layers)):
    print("layer", i, ":", len(layers[i]))
    
GAME.leaves_payoff(layers)

for ptr in layers[-2]:
    node = Node.MEM[ptr]
    leaf = Node.MEM[node.children[0, 0]]
    node.u, node.v = leaf.u, leaf.v

print("")
for layer in layers[-3::-1]:
    for ptr in layer:
        node = Node.MEM[ptr]
        M, N = len(node.actA), len(node.actB)
        A = np.empty([M, N])
        B = np.empty([M, N])
        for i in range(M):
            for j in range(N):
                A[i, j] = Node.MEM[node.children[i, j]].u
                B[i, j] = Node.MEM[node.children[i, j]].v
        NashX, NashY = supp_enum.supp_enum(A + 100, B + 100)
        # temporarily: what if 2 Nash eauilibria?
        X = NashX[0]
        Y = NashY[0]
        u = X @ A @ Y.T
        v = X @ B @ Y.T
        node.u, node.v = u, v
        print("node.h():", node.h(), "\nnode:", node, "\nA:\n", A, "\nB:\n", B, "\nNashX:", NashX, "\nNashY:", NashY)
        print("\n")

layer 0 : 1
layer 1 : 9
layer 2 : 36
layer 3 : 36

node.h(): [[1, 1]] 
node: {&: 2,
parent: 1,
just_happened: [1 1],
u: 1.38,
v: 2.95,
actA: [2 3],
actB: [2 3],
x: [0.8892 0.1108],
y: [0.3536 0.6464],
children:
[[11 12]
 [13 14]]} 
A:
 [[ 0.   -1.38]
 [ 1.38  0.  ]] 
B:
 [[ 0.   -2.95]
 [ 2.95  0.  ]] 
NashX: [[0. 1.]] 
NashY: [[1. 0.]]


node.h(): [[2, 1]] 
node: {&: 3,
parent: 1,
just_happened: [1 2],
u: 3.636460396039621,
v: -1.0383246618106279,
actA: [2 3],
actB: [1 3],
x: [0.0518 0.9482],
y: [0.3591 0.6409],
children:
[[15 16]
 [17 18]]} 
A:
 [[3.67 3.62]
 [6.38 2.29]] 
B:
 [[ 2.16 -7.07]
 [-1.17 -0.79]] 
NashX: [[0.0395 0.9605]] 
NashY: [[0.3292 0.6708]]


node.h(): [[3, 1]] 
node: {&: 4,
parent: 1,
just_happened: [1 3],
u: 1.8521534653465332,
v: 3.3138917793964646,
actA: [2 3],
actB: [1 2],
x: [0.8492 0.1508],
y: [0.4934 0.5066],
children:
[[19 20]
 [21 22]]} 
A:
 [[0.96 2.29]
 [3.67 0.96]] 
B:
 [[ 5.49 -0.79]
 [ 2.16  5.49]] 
NashX: [[0.3465 0.6535]] 
NashY: [[0.3292 0.6708]]



C:\Dev\supp_enum.py:44: RuntimeWarning: divide by zero encountered in divide
  y_J /= y_J.sum(axis=0)
